In [1]:
import cv2
import threading
import numpy as np
import tkinter as tk
import tensorflow as tf

from tkinter import Label
from collections import deque
from PIL import Image, ImageTk
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

In [2]:
# Ladda de förtränade modellerna
age_gender_model = load_model('age_gender_detection_model.keras')
emotion_model = load_model('improved_emotion_detection_model.keras')

# Emotion kategorier
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Färgkarta för emotioner (BGR-format)
emotion_colors = {
    'Angry': (0, 0, 255),
    'Disgust': (0, 255, 0),
    'Fear': (255, 0, 0),
    'Happy': (0, 255, 255),
    'Sad': (255, 255, 0),
    'Surprise': (255, 0, 255),
    'Neutral': (200, 200, 200)
}

# För att behålla senaste kända ansikte
last_face = None
frames_since_last_face = 0

In [3]:
def categorize_age(predicted_age):
    if predicted_age < 6:
        return "0-5"
    elif predicted_age < 13:
        return "6-12"
    elif predicted_age < 19:
        return "13-18"
    elif predicted_age < 26:
        return "19-25"
    elif predicted_age < 36:
        return "26-35"
    elif predicted_age < 46:
        return "36-45"
    elif predicted_age < 56:
        return "46-55"
    elif predicted_age < 66:
        return "56-65"
    elif predicted_age < 76:
        return "66-75"
    elif predicted_age < 86:
        return "76-85"
    elif predicted_age < 101:
        return "86-100"
    else:
        return "100+"

In [4]:
def predict_age_gender(face):
    face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face_gray = cv2.resize(face_gray, (64, 64))
    face_gray = np.expand_dims(face_gray, axis=-1)
    face_gray = np.expand_dims(face_gray, axis=0)

    age_prediction, gender_prediction = age_gender_model.predict(face_gray)
    predicted_age = age_prediction[0][0]
    predicted_age_range = categorize_age(predicted_age)
    
    # Här returnerar vi sannolikheten för att vara man istället för ett direkt beslut
    return predicted_age_range, gender_prediction[0][0]

def predict_emotion(face):
    face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face_gray = cv2.resize(face_gray, (48, 48))
    face_gray = face_gray.astype('float32') / 255.0
    face_gray = img_to_array(face_gray)
    face_gray = np.expand_dims(face_gray, axis=0)

    emotion_prediction = emotion_model.predict(face_gray)
    emotion_label = emotion_labels[np.argmax(emotion_prediction)]
    
    return emotion_label

In [5]:
def most_common(lst):
    return max(set(lst), key=lst.count)

def draw_text_with_border(frame, text, position, font, font_scale, text_color, border_color, thickness=2):
    x, y = position

    # Rita texten med kantlinje genom att rita samma text flera gånger i olika positioner
    cv2.putText(frame, text, (x - thickness, y), font, font_scale, border_color, thickness + 1)
    cv2.putText(frame, text, (x + thickness, y), font, font_scale, border_color, thickness + 1)
    cv2.putText(frame, text, (x, y - thickness), font, font_scale, border_color, thickness + 1)
    cv2.putText(frame, text, (x, y + thickness), font, font_scale, border_color, thickness + 1)

    cv2.putText(frame, text, (x - thickness, y - thickness), font, font_scale, border_color, thickness + 1)
    cv2.putText(frame, text, (x + thickness, y - thickness), font, font_scale, border_color, thickness + 1)
    cv2.putText(frame, text, (x - thickness, y + thickness), font, font_scale, border_color, thickness + 1)
    cv2.putText(frame, text, (x + thickness, y + thickness), font, font_scale, border_color, thickness + 1)

    # Rita den huvudsakliga texten ovanpå kantlinjen
    cv2.putText(frame, text, position, font, font_scale, text_color, thickness)


# Initiera köer för att hålla förutsägelser över tid
age_queue = deque(maxlen=10)
gender_prob_queue = deque(maxlen=10)
emotion_queue = deque(maxlen=10)

In [38]:
# Starta webbkameran
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Could not open webcam")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        frames_since_last_face = 0
        last_face = faces[0]  # Använd den första detekterade ansiktet
    else:
        frames_since_last_face += 1

    if last_face is not None and frames_since_last_face < 10:
        (x, y, w, h) = last_face
        face = frame[y:y+h, x:x+w]

        # Förutsäg ålder och kön
        predicted_age_range, gender_prob = predict_age_gender(face)
        age_queue.append(int(predicted_age_range.split('-')[0]))
        gender_prob_queue.append(gender_prob)

        # Förutsäg emotion och lägg till i kön
        predicted_emotion = predict_emotion(face)
        emotion_queue.append(predicted_emotion)

        # Beräkna stabiliserade resultat
        stable_age_numeric = np.mean(age_queue)
        stable_gender_prob = np.mean(gender_prob_queue)

        # Omvandla tillbaka från numeriskt värde till kategori
        stable_age_range = categorize_age(stable_age_numeric)
        stable_gender_str = 'Man' if stable_gender_prob < 0.1 else 'Woman'

        # Stabilisering av emotion
        stable_emotion = most_common(list(emotion_queue))

        # Hämta färgen för den stabiliserade emotionen
        color = emotion_colors.get(stable_emotion, (255, 255, 255))  # Default till vit om emotion inte hittas

        # Exempel på hur man använder draw_text_with_border
        draw_text_with_border(frame, f"Emotion: {stable_emotion}", (x, y-60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, (0, 0, 0), thickness=1)
        draw_text_with_border(frame, f"Age: {stable_age_range}", (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, (0, 0, 0), thickness=1)
        draw_text_with_border(frame, f"Gender: {stable_gender_str}", (x, y-90), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, (0, 0, 0), thickness=1)


    cv2.imshow('Age, Gender, and Emotion Detector', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━